In [1]:
import os
import h2o
import pandas as pd
import numpy as np
import datetime as dt
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2OGradientBoostingEstimator

In [70]:
h2o.cluster().shutdown()

In [55]:
h2o.init(nthreads = -1, max_mem_size = 25)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_131"; OpenJDK Runtime Environment (build 1.8.0_131-8u131-b11-2ubuntu1.16.04.3-b11); OpenJDK 64-Bit Server VM (build 25.131-b11, mixed mode)
  Starting server from /home/laith/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpt225otbw
  JVM stdout: /tmp/tmpt225otbw/h2o_laith_started_from_python.out
  JVM stderr: /tmp/tmpt225otbw/h2o_laith_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster version:,3.14.0.7
H2O cluster version age:,18 days
H2O cluster name:,H2O_from_python_laith_a5tqim
H2O cluster total nodes:,1
H2O cluster free memory:,22.22 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [66]:
def prepare_data(with_validation = True):
    train = h2o.import_file(os.path.realpath("../data/train2.csv"))
    test  = h2o.import_file(os.path.realpath("../data/test2.csv"))
    
    x = train.columns
    y = "target"
    x.remove(y)
    
    test_id = test['id'].as_data_frame(True).id.values
    
    features_bin = [i for i in x if set(i)&set('bin')==set('bin')]
    features_cat = [i for i in x if (set(i)&set('cat')==set('cat'))&(set(i)&set('avg')!=set('avg'))]
    features_con = list(set(x) - (set(features_bin)|set(features_cat)))
    
    train[features_cat] = train[features_cat].asfactor()
    train[features_bin] = train[features_bin].asfactor()
    train[features_con] = train[features_con].asnumeric()
    
    test[features_cat] = test[features_cat].asfactor()
    test[features_bin] = test[features_bin].asfactor()
    test[features_con] = test[features_con].asnumeric()
    
    train['target'] = train['target'].asfactor()
    
    return train, test, x, y, test_id
train, test, x, y, test_id = prepare_data()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


## Baseline

In [4]:
#We only provide the required parameters, everything else is default
gbm = H2OGradientBoostingEstimator(balance_classes=False,
                                 fold_assignment='stratified',
                                 nfolds=5,
                                 seed=41)
gbm.train(x=x, y=y, training_frame=train)

## Show a detailed model summary
print (gbm)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1510054492941_2033


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.03430890177728425
RMSE: 0.1852266227551651
LogLoss: 0.1491117313039438
Mean Per-Class Error: 0.38121237083209836
AUC: 0.6666474629804862
Gini: 0.3332949259609723
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.06471010073964742: 


,0,1,Error,Rate
0,540223.0,33295.0,0.0581,(33295.0/573518.0)
1,17378.0,4316.0,0.8011,(17378.0/21694.0)
Total,557601.0,37611.0,0.0851,(50673.0/595212.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0647101,0.1455527,197.0
max f2,0.0435855,0.2162397,260.0
max f0point5,0.0979174,0.1573262,142.0
max accuracy,0.2656676,0.9637423,56.0
max precision,0.8886166,1.0,0.0
max recall,0.0132927,1.0,398.0
max specificity,0.8886166,1.0,0.0
max absolute_mcc,0.0979174,0.1091963,142.0
max min_per_class_accuracy,0.0356138,0.6154697,295.0
max mean_per_class_accuracy,0.0366024,0.6187876,290.0


Gains/Lift Table: Avg response rate:  3.64 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2017-11-07 17:49:38,1 min 40.060 sec,0.0,0.1874009,0.1564850,0.5,1.0,0.9635525
,2017-11-07 17:49:39,1 min 40.615 sec,1.0,0.1872380,0.1557060,0.6087602,3.4966574,0.1230620
,2017-11-07 17:49:39,1 min 40.980 sec,2.0,0.1871068,0.1551183,0.6207162,3.6003701,0.1273983
,2017-11-07 17:49:40,1 min 41.377 sec,3.0,0.1869962,0.1546510,0.6237172,3.6446991,0.1111671
,2017-11-07 17:49:40,1 min 41.804 sec,4.0,0.1869039,0.1542724,0.6258451,3.7251114,0.1228823
,2017-11-07 17:49:41,1 min 42.409 sec,5.0,0.1868190,0.1539352,0.6288624,3.7877073,0.1408087
,2017-11-07 17:49:41,1 min 43.160 sec,6.0,0.1867411,0.1536368,0.6313536,3.9424662,0.1152732
,2017-11-07 17:49:42,1 min 43.954 sec,7.0,0.1866585,0.1533278,0.6351282,4.1300100,0.1189459
,2017-11-07 17:49:49,1 min 50.470 sec,33.0,0.1856165,0.1501007,0.6586623,5.6781501,0.1009909
,2017-11-07 17:49:54,1 min 55.584 sec,50.0,0.1852266,0.1491117,0.6666475,6.3556566,0.0851344


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
ps_reg_01_plus_ps_car_04_cat,547.0599976,1.0,0.2628650
ps_car_13,271.5241699,0.4963334,0.1304687
ps_ind_05_cat,216.9615173,0.3965955,0.1042511
ps_ind_17_bin,116.7248306,0.2133675,0.0560868
ps_car_06_cat,114.9836044,0.2101846,0.0552502
---,---,---,---
ps_car_04_cat_avg,0.0,0.0,0.0
ps_car_02_cat_avg,0.0,0.0,0.0
ps_car_08_cat_avg,0.0,0.0,0.0
ps_reg_01_plus_ps_car_02_cat_avg,0.0,0.0,0.0



See the whole table with table.as_data_frame()



## Tune

In [21]:
import math
# create hyperameter and search criteria lists (ranges are inclusive..exclusive))
hyper_params_tune = {'max_depth' : list(range(22,27,1)),
                'sample_rate': [x/100. for x in range(20,101)],
                'col_sample_rate' : [x/100. for x in range(50,101)],
                'col_sample_rate_per_tree': [x/100. for x in range(60,101)],
                'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(train.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,10)],
                'nbins_cats': [2**x for x in range(8,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["QuantilesGlobal","RoundRobin"]}
search_criteria_tune = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 60*60*2,  ## limit the runtime to 60 minutes
                   'max_models': 100,  ## build no more than 100 models
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "AUC",
                   'stopping_tolerance': 1e-3
                   }

In [22]:
gbm_final_grid = H2OGradientBoostingEstimator(distribution='bernoulli',
                    ## more trees is better if the learning rate is small enough 
                    ## here, use "more than enough" trees - we have early stopping
                    ntrees=10000,
                    ## smaller learning rate is better
                    ## since we have learning_rate_annealing, we can afford to start with a 
                    #bigger learning rate
                    learn_rate=0.05,
                    ## learning rate annealing: learning_rate shrinks by 1% after every tree 
                    ## (use 1.00 to disable, but then lower the learning_rate)
                    learn_rate_annealing = 0.99,
                    ## score every 10 trees to make early stopping reproducible 
                    #(it depends on the scoring interval)
                    score_tree_interval = 10,
                    ## fix a random number generator seed for reproducibility
                    seed = 1234,
                    ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
                    #5 consecutive scoring events
                    stopping_rounds = 5,
                    stopping_metric = "AUC",
                    stopping_tolerance = 1e-4)
            
#Build grid search with previously made GBM and hyper parameters
final_grid = H2OGridSearch(gbm_final_grid, 
                           hyper_params = hyper_params_tune,
                           grid_id = 'final_grid',
                          search_criteria = search_criteria_tune)
#Train grid search
final_grid.train(x=x, y=y, training_frame=train)

sorted_final_grid = final_grid.get_grid(sort_by='auc',decreasing=True)
print(sorted_final_grid)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%
    col_sample_rate col_sample_rate_change_per_level col_sample_rate_per_tree  \
0              0.83                             0.98                     0.68   
1              0.96                             1.07                      0.6   
2              0.65                             1.02                     0.77   
3              0.56                             0.99                      0.6   
4              0.88                             1.02                     0.78   
5              0.63                              1.0                     0.65   

    histogram_type max_depth min_rows min_split_improvement nbins nbins_cats  \
0       RoundRobin        25      4.0                1.0E-8   512       4096   
1       RoundRobin        24     64.0                1.0E-6   128       4096   
2       RoundRobin        24   1024.0                1.0E-8    64        256   
3  QuantilesGlobal        24   

## Final grid results

In [33]:
my_gbm = H2OGradientBoostingEstimator(distribution="bernoulli",
                                      ntrees=380,
                                      learn_rate=0.07,
                                      learn_rate_annealing = 0.99,
                                      score_tree_interval = 10,
                                      max_depth=25,
                                      min_rows=4.0,
                                      sample_rate=0.87,
                                      col_sample_rate=0.83,
                                      col_sample_rate_per_tree=0.68,
                                      col_sample_rate_change_per_level=0.98,
                                      nbins=512,
                                      nbins_cats=4096,
                                      min_split_improvement=1.0E-8,
                                      histogram_type='RoundRobin',
                                      nfolds=3,
                                      fold_assignment="stratified",
                                      #keep_cross_validation_predictions=True,
                                      seed=1234,
                                      stopping_rounds = 5,
                                      stopping_metric = "AUC",
                                      stopping_tolerance = 1e-4)
my_gbm.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [56]:
my_gbm.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid
0,accuracy,0.7534814,0.029731976,0.7335403,0.7149366,0.81196725
1,auc,0.575426,0.0018294846,0.577219,0.5717672,0.57729167
2,err,0.24651863,0.029731976,0.26645973,0.2850634,0.18803276
3,err_count,48924.332,5955.19,52791.0,56747.0,37235.0
4,f0point5,0.060853146,0.001052654,0.06133346,0.05883782,0.062388152
5,f1,0.08717165,8.765584E-4,0.088820614,0.08583166,0.08686269
6,f2,0.15415972,0.005653397,0.16095333,0.1585903,0.14293554
7,lift_top_group,2.155823,0.106553115,2.3588893,1.9983099,2.1102698
8,logloss,0.25604457,0.0039391364,0.25996655,0.2600008,0.24816631
9,max_per_class_error,0.67780733,0.035854846,0.64902467,0.6353183,0.74907905


In [67]:
test

ps_car_13,ps_reg_03,ps_ind_05_cat,ps_ind_03,ps_ind_15,ps_reg_02,ps_car_14,ps_car_12,ps_car_01_cat,ps_car_07_cat,ps_ind_17_bin,ps_car_03_cat,ps_reg_01,ps_car_15,ps_ind_01,ps_ind_16_bin,ps_ind_07_bin,ps_car_06_cat,ps_car_04_cat,ps_ind_06_bin,ps_car_09_cat,ps_car_02_cat,ps_ind_02_cat,ps_car_11,ps_car_05_cat,ps_calc_09,ps_calc_05,ps_ind_08_bin,ps_car_08_cat,ps_ind_09_bin,ps_ind_04_cat,ps_ind_18_bin,ps_ind_12_bin,ps_ind_14,ps_reg_01_plus_ps_car_02_cat,ps_reg_01_plus_ps_car_04_cat,ps_ind_05_cat_avg,ps_car_01_cat_avg,ps_car_07_cat_avg,ps_car_03_cat_avg,ps_car_06_cat_avg,ps_car_04_cat_avg,ps_car_09_cat_avg,ps_car_02_cat_avg,ps_ind_02_cat_avg,ps_car_05_cat_avg,ps_car_08_cat_avg,ps_ind_04_cat_avg,ps_reg_01_plus_ps_car_02_cat_avg,ps_reg_01_plus_ps_car_04_cat_avg,id
0.669556,0.610328,0,8,12,0.3,0.352136,0.316228,7,1,0,,0.5,3.4641,0,1,1,1,0,0,2,1,1,1,,2,1,0,1,0,1,0,0,0,14,50,0.033865,0.0287704,0.0347665,0.0364475,0.0341848,0.0334507,0.0363187,0.0337719,0.0357246,0.0364475,0.034735,0.0386438,0.0300484,0.028365,7
0.60632,0.771362,0,5,5,0.5,0.358329,0.316228,4,1,0,,0.9,2.82843,4,1,0,11,0,0,0,1,2,1,0,2,3,0,1,1,1,0,0,0,23,90,0.033865,0.039161,0.0347665,0.0364475,0.0317197,0.0334507,0.033344,0.0337719,0.0381394,0.0401105,0.034735,0.0386438,0.0382137,0.0383473,9
0.896239,0.916174,0,3,10,0,0.398497,0.4,11,1,0,,0.4,3.31662,5,0,0,14,0,0,2,1,1,3,,3,3,0,1,1,0,0,0,0,11,40,0.033865,0.0415806,0.0347665,0.0364475,0.0342936,0.0334507,0.0363187,0.0337719,0.0357246,0.0364475,0.034735,0.0347903,0.030893,0.0305486,13
0.65211,nan,0,6,4,0.2,0.381445,0.374166,7,1,0,,0.1,2.44949,0,1,0,1,0,1,2,1,1,2,,1,1,0,1,0,0,0,0,0,3,10,0.033865,0.0287704,0.0347665,0.0364475,0.0341848,0.0334507,0.0363187,0.0337719,0.0357246,0.0364475,0.034735,0.0347903,0.0228512,0.0201968,16
0.812914,0.817771,0,7,4,0.4,0.385097,0.374166,11,1,0,,0.9,3.31662,5,1,0,11,0,0,2,1,1,3,,4,4,0,1,1,0,0,0,0,23,90,0.033865,0.0415806,0.0347665,0.0364475,0.0317197,0.0334507,0.0363187,0.0337719,0.0357246,0.0364475,0.034735,0.0347903,0.0382137,0.0383473,17
0.750922,0.759934,0,6,10,0.5,0.368782,0.316228,9,0,0,,0.9,3.60555,0,1,0,11,0,1,2,1,1,2,,4,1,0,0,0,0,0,0,0,23,90,0.033865,0.0544211,0.0511856,0.0364475,0.0317197,0.0334507,0.0363187,0.0337719,0.0357246,0.0364475,0.0449334,0.0347903,0.0382137,0.0383473,19
0.724569,nan,0,3,11,0.1,0.381445,0.316228,6,1,1,,0.1,3.31662,0,0,1,1,0,0,0,1,1,2,0,5,1,0,1,0,0,0,0,0,3,10,0.033865,0.029763,0.0347665,0.0364475,0.0341848,0.0334507,0.033344,0.0337719,0.0357246,0.0401105,0.034735,0.0347903,0.0228512,0.0201968,20
0.58661,1.14592,0,0,7,1.1,0.358329,0.316228,7,1,0,,0.9,2.64575,0,1,0,11,0,1,2,1,1,3,,2,2,0,1,0,0,0,0,0,23,90,0.033865,0.0287704,0.0347665,0.0364475,0.0317197,0.0334507,0.0363187,0.0337719,0.0357246,0.0364475,0.034735,0.0347903,0.0382137,0.0383473,22
1.64015,0.59739,0,7,6,0,0.367423,0.447214,11,,0,1,0.4,3.74166,0,1,1,2,1,0,0,0,1,2,0,0,2,0,0,0,0,0,0,0,10,41,0.033865,0.0415806,0.0364475,0.0482978,0.0659614,0.0488245,0.033344,0.0495075,0.0357246,0.0401105,0.0449334,0.0347903,0.055974,0.0523091,26
1.04134,nan,0,6,7,1,0.422493,0.447214,11,1,1,,0.9,3.4641,1,0,0,4,0,0,2,0,1,2,,1,1,0,1,1,0,0,0,0,22,90,0.033865,0.0415806,0.0347665,0.0364475,0.0338515,0.0334507,0.0363187,0.0495075,0.0357246,0.0364475,0.034735,0.0347903,0.0518859,0.0383473,28


In [68]:
filled = test.fillna(method="forward",axis=0,maxlen=1)

In [69]:
my_gbm.predict(filled)

KeyboardInterrupt: 

In [57]:
sub = pd.DataFrame()
sub['id'] = test_id
sub['target'] = np.zeros_like(test_id)
p = my_gbm.predict(test_data=filled)
sub['target'] = p['p1'].as_data_frame(True).values
filename = 'GBM_model_'+str(dt.datetime.now()).replace(' ','_').replace(':','').replace('.','')
sub.id = sub.id.astype('Int32')
sub.to_csv('../output/'+filename+'.csv', index=False)

H2OResponseError: Server error water.exceptions.H2OKeyNotFoundArgumentException:
  Error: Object 'GBM_model_python_1510130641569_291' not found in function: predict for argument: model
  Request: POST /4/Predictions/models/GBM_model_python_1510130641569_291/frames/py_24_sid_9564


## Optimized

In [9]:
my_gbm = H2OGradientBoostingEstimator(distribution="bernoulli",
                                      ntrees=100,
                                      max_depth=27,
                                      min_rows=2048,
                                      learn_rate=0.2,
                                      sample_rate=0.37,
                                      col_sample_rate=0.63,
                                      col_sample_rate_per_tree=0.85,
                                      col_sample_rate_change_per_level=1.0,
                                      nbins=128,
                                      nbins_cats=2096,
                                      min_split_improvement=0.0,
                                      histogram_type='RoundRobin',
                                      nfolds=5,
                                      fold_assignment="Modulo",
                                      keep_cross_validation_predictions=True,
                                      seed=1)
my_gbm.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [10]:
my_gbm.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.82658774,0.022030594,0.8682073,0.8345724,0.8427194,0.8115623,0.7758774
1,auc,0.6175691,0.0024692148,0.6171947,0.62397945,0.61585146,0.61734164,0.61347824
2,err,0.17341225,0.022030594,0.13179271,0.16542763,0.15728062,0.1884377,0.22412258
3,err_count,20643.4,2622.5352,15689.0,19693.0,18723.0,22432.0,26680.0
4,f0point5,0.07812229,0.00207067,0.081190914,0.08007955,0.07857658,0.07806611,0.07269828
5,f1,0.10674676,0.0014466541,0.10609082,0.109276764,0.10617272,0.10863864,0.10355487
6,f2,0.16942798,0.0070918794,0.1530193,0.17198178,0.16364492,0.17857143,0.17992248
7,lift_top_group,3.1388187,0.14692552,3.4482324,3.2725856,3.129951,2.8590188,2.984306
8,logloss,0.1544154,6.154764E-4,0.15310773,0.15372127,0.15466785,0.15532508,0.15525506
9,max_per_class_error,0.7163328,0.033193067,0.78298366,0.7214664,0.7439558,0.68718535,0.64607257
